# Skin Cancer Classification Challenge

**Summary : ** skin cancer.

## Preliminaries and Imports

In [1]:
#general & system\n",
import os

#data processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = 10, 10  #default setting
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#data augmentation
from PIL import Image
from random import choice
import cv2
import numpy as np
#import keras.preprocessing.image as prep

#ML part
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image as kimage
from keras.models import Sequential
from keras.layers import GlobalMaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Dropout,Activation,Flatten
from keras.models import Model
from keras.optimizers import SGD
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

#validation
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split

%matplotlib inline

Using TensorFlow backend.


## I. Load Data

- **Rq : ** Datasets are quite light, can be fully loaded in a laptop memory with ease.

In [2]:
%%time
meta = pd.read_csv("data/metadata.csv")

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.34 ms


In [3]:
%%time
filelist = os.listdir("data/resized/")
X_train = np.array([np.array(Image.open("data/resized/"+fname)) for fname in filelist])

CPU times: user 688 ms, sys: 36 ms, total: 724 ms
Wall time: 722 ms


In [4]:
X_train.shape

(617, 224, 224, 3)

In [5]:
#get target
filelist = [x.replace(".jpg","") for x in filelist]
Y_train = [meta[meta["name"]==x]["meta_clinical_benign_malignant"].values[0] for x in filelist]
Y_train = [1 if x=="malignant" else 0 for x in Y_train]

In [6]:
#control
print(filelist[0:5])
print(Y_train[0:5])

['ISIC_0000256', 'ISIC_0000282', 'ISIC_0000153', 'ISIC_0000225', 'ISIC_0001150']
[0, 1, 1, 0, 0]


## II. Data Preparation

In [7]:
#data splitting
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, random_state=42, train_size=0.80)

## III. Deep Learning with Transfer Learning

In [8]:
weights_path = 'model/checkpoints/'

if not os.path.exists(weights_path):
    os.makedirs(weights_path)
    
epochs = 500
batch_size = 16

- **Using VGG16 : **

In [9]:
#load base model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [10]:
#adding top layers
    #sequentialy
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:])) #Flatten/GlobalAveragePooling2D
add_model.add(Dense(1024, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(512, activation='relu'))
add_model.add(Dropout(0.25))
add_model.add(Dense(1, activation='sigmoid'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [11]:
#freeze lower layers of the model
#for layer in model.layers[:]:
    #layer.trainable = False
for layer in model.layers[0:]:
    layer.trainable = True
    
#compile
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
%%time
#Lets define the image transormations that we wan

gen = ImageDataGenerator(horizontal_flip=True,
                         vertical_flip=True,
                         width_shift_range=0.2,
                         height_shift_range=0.2,
                         zoom_range=0.2,
                         rotation_range=40)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_one_input(X1, y):
    genX1 = gen.flow(X1, y, batch_size=batch_size, seed=42)
    while True:
        X1i = genX1.next()
        yield X1i[0], X1i[1]

#Finally create out generator
gen_flow = gen_flow_for_one_input(X_train, Y_train)

from keras.callbacks import EarlyStopping, ModelCheckpoint
epochs_to_wait_for_improve = 50
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=epochs_to_wait_for_improve)
checkpoint_callback = ModelCheckpoint(weights_path + 'BestKerasModelVGG16_flat.h5', monitor='val_loss',
                                      verbose=1, save_best_only=True, mode='min')
#fit the model
model.fit_generator(gen_flow, validation_data=(X_valid, Y_valid),
                    steps_per_epoch=int(np.ceil(len(X_train)/batch_size)),
                    epochs=500, verbose=1, callbacks=[early_stopping_callback, checkpoint_callback])

Epoch 1/500
30/30 [==============================] - 38s 1s/step - loss: 1.1799 - acc: 0.6167 - val_loss: 0.6652 - val_acc: 0.6129
Epoch 2/500
30/30 [==============================] - 35s 1s/step - loss: 0.6466 - acc: 0.6990 - val_loss: 0.6697 - val_acc: 0.6129
Epoch 3/500
30/30 [==============================] - 31s 1s/step - loss: 0.6351 - acc: 0.6756 - val_loss: 0.6753 - val_acc: 0.6129
Epoch 4/500
30/30 [==============================] - 31s 1s/step - loss: 0.6060 - acc: 0.6944 - val_loss: 0.6788 - val_acc: 0.6129
Epoch 5/500
30/30 [==============================] - 31s 1s/step - loss: 0.5921 - acc: 0.7127 - val_loss: 0.6764 - val_acc: 0.6129
Epoch 6/500
30/30 [==============================] - 31s 1s/step - loss: 0.6269 - acc: 0.6742 - val_loss: 0.6700 - val_acc: 0.6129
Epoch 7/500
30/30 [==============================] - 31s 1s/step - loss: 0.6087 - acc: 0.6981 - val_loss: 0.6701 - val_acc: 0.6129
Epoch 8/500
30/30 [==============================] - 32s 1s/step - loss: 0.6003 - a

Epoch 32/500
30/30 [==============================] - 29s 972ms/step - loss: 0.5662 - acc: 0.7125 - val_loss: 0.6257 - val_acc: 0.6290
Epoch 33/500
30/30 [==============================] - 28s 948ms/step - loss: 0.5377 - acc: 0.7298 - val_loss: 0.6307 - val_acc: 0.6452
Epoch 34/500
30/30 [==============================] - 28s 950ms/step - loss: 0.5493 - acc: 0.7147 - val_loss: 0.6342 - val_acc: 0.6452
Epoch 35/500
30/30 [==============================] - 29s 967ms/step - loss: 0.5346 - acc: 0.7377 - val_loss: 0.6116 - val_acc: 0.6452
Epoch 36/500
30/30 [==============================] - 28s 949ms/step - loss: 0.5647 - acc: 0.7476 - val_loss: 0.6144 - val_acc: 0.6855
Epoch 37/500
30/30 [==============================] - 29s 953ms/step - loss: 0.5447 - acc: 0.7344 - val_loss: 0.6181 - val_acc: 0.6855
Epoch 38/500
30/30 [==============================] - 29s 969ms/step - loss: 0.5736 - acc: 0.7189 - val_loss: 0.5984 - val_acc: 0.6452
Epoch 39/500
30/30 [==============================] - 2

Epoch 61/500
30/30 [==============================] - 29s 957ms/step - loss: 0.4678 - acc: 0.7997 - val_loss: 0.4920 - val_acc: 0.7339
Epoch 62/500
30/30 [==============================] - 29s 975ms/step - loss: 0.4792 - acc: 0.8094 - val_loss: 0.4813 - val_acc: 0.7581
Epoch 63/500
30/30 [==============================] - 29s 953ms/step - loss: 0.4927 - acc: 0.7688 - val_loss: 0.5843 - val_acc: 0.6613
Epoch 64/500
30/30 [==============================] - 28s 948ms/step - loss: 0.4990 - acc: 0.7710 - val_loss: 0.5485 - val_acc: 0.7258
Epoch 65/500
30/30 [==============================] - 29s 958ms/step - loss: 0.4700 - acc: 0.7835 - val_loss: 0.4845 - val_acc: 0.7581
Epoch 66/500
30/30 [==============================] - 28s 949ms/step - loss: 0.4647 - acc: 0.7773 - val_loss: 0.4958 - val_acc: 0.7661
Epoch 67/500
30/30 [==============================] - 28s 945ms/step - loss: 0.4869 - acc: 0.7756 - val_loss: 0.6298 - val_acc: 0.7177
Epoch 68/500
30/30 [==============================] - 2

Epoch 93/500
30/30 [==============================] - 29s 955ms/step - loss: 0.3557 - acc: 0.8699 - val_loss: 0.4812 - val_acc: 0.7500
Epoch 94/500
30/30 [==============================] - 29s 954ms/step - loss: 0.4189 - acc: 0.8104 - val_loss: 0.4610 - val_acc: 0.7903
Epoch 95/500
30/30 [==============================] - 29s 950ms/step - loss: 0.3284 - acc: 0.8771 - val_loss: 0.4765 - val_acc: 0.7581
Epoch 96/500
30/30 [==============================] - 28s 950ms/step - loss: 0.3566 - acc: 0.8511 - val_loss: 0.4670 - val_acc: 0.7500
Epoch 97/500
30/30 [==============================] - 28s 946ms/step - loss: 0.3423 - acc: 0.8553 - val_loss: 0.4571 - val_acc: 0.7742
Epoch 98/500
30/30 [==============================] - 29s 956ms/step - loss: 0.3691 - acc: 0.8423 - val_loss: 0.5023 - val_acc: 0.7742
Epoch 99/500
30/30 [==============================] - 29s 967ms/step - loss: 0.3406 - acc: 0.8574 - val_loss: 0.4274 - val_acc: 0.7984
Epoch 100/500
30/30 [==============================] - 

30/30 [==============================] - 29s 955ms/step - loss: 0.3042 - acc: 0.8583 - val_loss: 1.0667 - val_acc: 0.6694
Epoch 126/500
30/30 [==============================] - 28s 946ms/step - loss: 0.2098 - acc: 0.9224 - val_loss: 0.5110 - val_acc: 0.7581
Epoch 127/500
30/30 [==============================] - 29s 953ms/step - loss: 0.2414 - acc: 0.9037 - val_loss: 0.5985 - val_acc: 0.7823
Epoch 128/500
30/30 [==============================] - 28s 944ms/step - loss: 0.2584 - acc: 0.8923 - val_loss: 0.5450 - val_acc: 0.7581
Epoch 129/500
30/30 [==============================] - 28s 945ms/step - loss: 0.2620 - acc: 0.8928 - val_loss: 0.5268 - val_acc: 0.7581
Epoch 130/500
30/30 [==============================] - 28s 950ms/step - loss: 0.2384 - acc: 0.9099 - val_loss: 0.6875 - val_acc: 0.7742
Epoch 131/500
30/30 [==============================] - 29s 953ms/step - loss: 0.1896 - acc: 0.9208 - val_loss: 0.7285 - val_acc: 0.7419
Epoch 132/500
30/30 [==============================] - 28s 949

## IV. Evaluate Model

In [13]:
model = load_model(weights_path+"BestKerasModelVGG16_flat.h5")
score = model.evaluate(X_valid, Y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

ResourceExhaustedError: OOM when allocating tensor with shape[32,64,224,224]
	 [[Node: block1_conv2_1/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv1_1/Relu, block1_conv2_1/kernel/read)]]
	 [[Node: loss_1/mul/_923 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_274_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'block1_conv2_1/convolution', defined at:
  File "/home/paul/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/paul/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/paul/anaconda2/lib/python2.7/site-packages/ipykern

In [ ]:
#compute log loss
pred = model.predict(X_valid)
log_loss(Y_valid,pred)